In [1]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd
import numpy as np
import pandas
from IPython.display import display, HTML
pandas.set_option('display.max_colwidth', -1)
basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/'
vardir = basedir + 'variables/'
geodir = basedir + 'geography/'
datadir = basedir + 'data/'
errordir = basedir + 'error/'

extras_dir = '/home/idies/workspace/Storage/raddick/census/extras/'

# category can be one of: demographics, ancestry, residence, transportation... 
#      household, income, employment, housing, qa
category = 'demographics'

print('ok')

ok


In [18]:
print('Reading metadata...')
metadata_df = pandas.read_csv(vardir+'/variables_acs2017_{0:}.csv'.format(category), low_memory=False, index_col=0, encoding='utf-8')
metadata_df = metadata_df.drop('variable.1', axis=1)
geo_metadata_df = pandas.read_csv(geodir+'geo_variables_acs2017.csv', low_memory=False, index_col=0, encoding='utf-8')
geo_metadata_df = geo_metadata_df.set_index('variable')

print('Reading geography...')
geo_df = pandas.read_csv(geodir+'geo_acs2017.csv', low_memory=False, index_col=0, encoding='utf-8')

print('Reading data (estimates)...')
data_df = pandas.read_csv(datadir+'data_acs2017_{0:}.csv'.format(category), low_memory=False, index_col=0, encoding='utf-8')

print('Reading margins of error...')
error_df = pandas.read_csv(errordir+'error_acs2017_{0:}.csv'.format(category), low_memory=False, index_col=0, encoding='utf-8')

print('making backups...')
data_df_bk = data_df

print('Done!')


Reading metadata...
Reading geography...
Reading data (estimates)...
Reading margins of error...
making backups...
Done!


In [21]:
print('retrieving from backup...')
data_df = data_df_bk

print('Calculating geography sumary levels...')
#data_df['SUMLEVEL'] = data_df['GEOID'].apply(lambda x: int(x[0:3]))
data_df = data_df.assign(SUMLEVEL = data_df['GEOID'].apply(lambda x: int(x[0:3])))
#data_df

print('Documenting geography summary levels...')
sumlevel_df = pandas.read_csv(extras_dir+'geo_summary_levels.csv', encoding='utf-8')
sumlevel_df = sumlevel_df.set_index('SUMLEVEL')
sumlevel_df

retrieving from backup...
Calculating geography sumary levels...
Documenting geography summary levels...


,DESCRIPTION
SUMLEVEL,
10,United States
20,Region
30,Division
40,State
50,State-County
60,State-County-County Subdivision
67,State-County-County Subdivision-Subminor Civil Division
70,State-County-County Subdivision-Place/Remainder
80,State-County-County Subdivision-Place/Remainder-Census Tract


In [25]:
print('Documenting state codes')
geo_df[['STATE','STUSAB']].dropna().drop_duplicates().sort_values('STATE').to_csv(geodir+'statecodes.csv', encoding='utf-8', index=False)
statecodes_df = pandas.read_csv(geodir+'statecodes.csv', encoding='utf-8')
statecodes_df['STATE'] = statecodes_df['STATE'].astype('int')
statecodes_df = statecodes_df.set_index('STATE')
print('Done!')
##sumlevel_df

Documenting state codes
Done!


In [ ]:
#### metadata_df
#metadata_df[metadata_df['sequence_number'] == 150]
#metadata_df
#B26001_001
#GROUP QUARTERS POPULATION for Population In Group Quarters% Total

#string1 = 'Baltimore'
#string2 = ''
#string3 = ''

#print('{0:}'.format(geo_df['NAME'].loc['05000US24510']))
#print('{0:} {1:,.0f}'.format(metadata_df['description'].loc['B26001_001'], data_df['B26001_001'].loc['05000US24510']))

#data_df['B26001_001'].loc['05000US24510']


#data_df      
#05000US24510
#data_df['B26001_001'].sample(1)

phrases = []
phrases.append('Total Population')
#phrases.append('Female')
phrases.append('White')
#phrases.append('Male')

thelist = find_variables4(metadata_df, phrases)
#print('Variables found: ',thelist)




geos = ['Florida']
#wantlevels = [10, 160]
#wantlevels = [160]
wantlevels = [50]
#wantlevels = []
print('Finding matching geographies...')
geolist = find_geography4(geo_df, geos, wantlevels)
print('Retrieving data...')

#ourdata_df = get_data(data_df, thelist, geolist)
#ourdata_df = ourdata_df.join(data_df['NAME'])
#ourdata_df
metadata_df[metadata_df['variable'].apply(lambda x: x in thelist)]

In [ ]:
def find_geography4(gdf, tofindlist, wantlevels = []):
    geosub_df = pandas.DataFrame()
    geolist = []

    if (len(tofindlist) == 0):
        print('CAUTION: No geography search phrases specified, returning every geography!')
    if (len(tofindlist) > 4):
        print('ERROR: list contains more than four geography search phrases, searching for only the first three')
    try:
        string1 = tofindlist[0]
    except IndexError:
        string1 = ''
    try:
        string2 = tofindlist[1]
    except IndexError:
        string2 = ''
    try:
        string3 = tofindlist[2]
    except IndexError:
        string3 = ''
    try:
        string4 = tofindlist[3]
    except IndexError:
        string4 = ''
    
    # If wantlevels not specified, search through every level
    if (len(wantlevels) == 0):
        geosub_df = gdf
    for thislevel in wantlevels: 
        if (not(np.isnan(thislevel))):
            geosub_df = pandas.concat((geosub_df,gdf[gdf['SUMLEVEL'] == thislevel]))
#            print(geosub_df.head(1))
    found_df = geosub_df[geosub_df['NAME'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x) & (string4 in x))]
    geolist = found_df['GEOID'].values.tolist()

    return geolist
    
def warn_geo_level():
    
    helpshow_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geography/geo_summary_levels.csv')
    helpshow_df = helpshow_df.set_index('SUMLEVEL')

    showme = '<table><tr><th>SUMLEVEL</th><th>Description</th><th>Count</th></tr>'
    for slvl, row in helpshow_df.iterrows():
        showme += '<tr><td>'+str(slvl)+'</td><td>'+str(row['description'])+'</td><td>'+str(row['acscount'])+'</td></tr>'
    showme += '</table>'

    return showme
    
def find_variables4(mdf, tofindlist):
    # NEED TO REWRITE TO USE REGEX INSTEAD, because now only "Male" returns men; "male" returns women b/c feMALE
    varlist = []
    if (len(tofindlist) == 0):
        print('CAUTION: No search phrases specified, returning every variable!')
    if (len(tofindlist) > 4):
        print('ERROR: list contains more than four search phrases, searching for only the first three')
    try:
        string1 = tofindlist[0]
    except IndexError:
        string1 = ''
    try:
        string2 = tofindlist[1]
    except IndexError:
        string2 = ''
    try:
        string3 = tofindlist[2]
    except IndexError:
        string3 = ''
    try:
        string4 = tofindlist[3]
    except IndexError:
        string4 = ''
            
    varlist = mdf['variable'][mdf['description'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x) & (string4 in x))].values.tolist()
    return varlist

def get_data(df, varlist = [], geolist = []):

    rdf = pandas.DataFrame()
    
    if (len(geolist) == 0):
        print('Caution: No geographies specified, will return all geographies')
        rdf = df
    else:
        for thisgeo in geolist:
            try:
                rdf = rdf.append(df.loc[thisgeo])
            except:
                print('Geography {0:} not found. Skipping.'.format(thisgeo))

    if (len(varlist) == 0):
        print('CAUTION: No variables specified, will return all variables')
    else:
        vars_that_exist = ['GEOID']
        for thisvar in varlist:
            if (thisvar in df.columns):
                vars_that_exist.append(thisvar)
            else:
                print('Variable {0:} not found. Skipping.'.format(thisvar))
        rdf = rdf[vars_that_exist]
    rdf.index.name = ''

    return rdf
    
print('Done')

In [ ]:
#sumlevel_df

In [ ]:
#geo_df[geo_df['SUMLEVEL'] != 40].groupby('STATE').size()
# US = 1 if this is data for the U.S., only for SUMLEVEL = 10
# REGION from 1 to 4, only for SUMLEVEL = 20
# DIVISION from 1 to 9, only for SUMLEVEL = 30
# STATE from 1 (AL) to 72 (PR), but skips several values. But they map one-to-one with STUSAB, so we don't really need them.
# COUNTY from 1 to 810, different for different states. Numbers mostly alphabetical list, but with skips (old names?)

# PLACE has only 596 distinct values, they seem mostly random
# CBSA is probably combined statistical area, seems mostly random

#geo_df[['STATE','STUSAB','NAME']][geo_df['STATE'] == 3].sample(10)
#geo_df.groupby('PLACE').size()
geo_df[['SUMLEVEL','STUSAB','PLACE','NAME']][geo_df['SUMLEVEL'] == 170]

In [ ]:
data_df['SUMLEVEL'] = data_df['GEOID'].apply(lambda x: int(x[0:3]))

print('Estimated U.S. population: {0:,.0f}'.format(data_df['B01001_001'].loc['01000US']))

print('For the four census regions: {0:,.0f}'.format(data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US'].sum()))
print('\n')
print('Population by census region:')
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US1'], data_df['B01001_001'].loc['02000US1']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US2'], data_df['B01001_001'].loc['02000US2']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US3'], data_df['B01001_001'].loc['02000US3']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US4'], data_df['B01001_001'].loc['02000US4']))

#print('Men: {0:,.0f} (error: {1:,.0f})'.format(data_df['B01001_002'].loc['01000US'], error_df['B01001_002'].loc['01000US']))
#print('Women: {0:,.0f} (error: {1:,.0f})'.format(data_df['B01001_026'].loc['01000US'], error_df['B01001_026'].loc['01000US']))

#print('Margin of error: {0:,.0f}'.format(error_df['B01001_001'].loc['01000US']))

In [ ]:
#data_df[data_df['SUMLEVEL'] == 40]

display(HTML('<h2>Population by state</h2>'))

statetable = '<table><tr><th>State</th><th>Population</th></tr>'
for thisgeoid, thisrow in data_df[data_df['GEOID'].apply(lambda x: x[0:5]) == '04000'].iterrows():
    statetable += '<tr>'
    statetable += '<td>{0:}</td><td style="text-align:right;padding:5px">{1:,.0f}</td>'.format(thisrow['NAME'], thisrow['B01001_001'])
    statetable += '</tr>'
statetable += '</table>'

display(HTML(statetable))

total_population = data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:5]) == '04000'].sum()

display(HTML(('TOTAL of all 50 states: <b>{0:,.0f}</b> (not equal to national b/c military etc. is federal)'.format(total_population))))
#print('For the four census regions: {0:,.0f}'.format(data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US'].sum()))
#data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US']



In [ ]:
#geo_df[['SUMLEVEL','STUSAB','COUNTY','NAME']][(geo_df['SUMLEVEL'] == 310) & (geo_df['STUSAB'] == 'MD')].sort_values('COUNTY')
#geo_df[geo_df['SUMLEVEL'] == 310].groupby('CBSA').size().sort_values()
geo_df.sample(1).T